In [ ]:
!pip install pymorphy3
!pip install transformers

In [1]:
import os # работа с папкой и файлами
import re # регулярные выражения, доп вариант к очистке
import string # работа со строкой
import math
import operator
import random
import time
import json
import datetime
import csv
import requests
import joblib

import pandas as pd
import numpy as np

from wordcloud import WordCloud # визуальное отображение
import pymorphy3 # работа с русским языком, pymorphy3

import matplotlib.pyplot as plt # визуальное отображение
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split # Деление выборки на тестовые и тренировочные данные

# python -m pip install transformers
from transformers import BertTokenizer # токенизатор BERT
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk # работа с пакетами языков
from nltk import word_tokenize, ngrams # токенизация и деление на n граммы
from nltk.corpus import stopwords # стопслова, extend

# python -m pip install tensorflow
# python -m pip install torch torchvision torchaudio
import tensorflow
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, f1_score # Критерий качества, точности

# скачаем словарь стоп-слов
nltk.download("stopwords")

# скачиваем модель, которая будет делить на предложения
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Просмотр файлов в директории
print(os.listdir("archive"))

['dev-v1.1.json', 'train-v1.1.json']


In [34]:
# Парсер файлов .json в DataFrame
def convert(f, path=['data', 'paragraphs', 'qas', 'answers']):
  file = json.loads(open(f).read())
  m = pd.json_normalize(file, path[:-1])
  r = pd.json_normalize(file, path[:-2])
  index = np.repeat(r['context'].values, r.qas.str.len())
  m['context'] = index
  data = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
  data['context_id'] = data['context'].factorize()[0]
  return data

# Тренировочные данные
train = convert('archive/train-v1.1.json')

# Валидационные данные
val = convert('archive/dev-v1.1.json')

train

,id,question,context,answers,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 515, 'text': 'Saint Bernadet...",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 188, 'text': 'a copper statu...",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 279, 'text': 'the Main Build...",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...","[{'answer_start': 381, 'text': 'a Marian place...",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...","[{'answer_start': 92, 'text': 'a golden statue...",0
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 229, 'text': 'Oregon'}]",18890
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 414, 'text': 'Rangoon'}]",18890
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 476, 'text': 'Minsk'}]",18890
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...","[{'answer_start': 199, 'text': '1975'}]",18890


In [32]:
# Размер тренировочных данных
print(f'Размер тренировочных данных: {train.shape}')

# Количество уникальных контекстов в наборе тренировочных данных
print(f'Количество уникальных контекстов в наборе тренировочных данных: {train["context_id"].unique().size}')

# Список уникальных контекстов в наборе тренировочных данных
train_unique = train[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
train_unique

Размер тренировочных данных: (87599, 5)
Количество уникальных контекстов в наборе тренировочных данных: 18891


,context,context_id
0,"Architecturally, the school has a Catholic cha...",0
1,"As at most other universities, Notre Dame's st...",1
2,The university is the major seat of the Congre...,2
3,The College of Engineering was established in ...,3
4,All of Notre Dame's undergraduate students are...,4
...,...,...
18886,"Institute of Medicine, the central college of ...",18886
18887,Football and Cricket are the most popular spor...,18887
18888,The total length of roads in Nepal is recorded...,18888
18889,The main international airport serving Kathman...,18889


In [33]:
# Размер валидационных данных
print(f'Размер валидационных данных: {val.shape}')

# Количество уникальных контекстов в наборе валидационных данных
print(f'Количество уникальных контекстов в наборе валидационных данных: {val["context_id"].unique().size}')

# Список уникальных контекстов в наборе валидационных данных
val_unique = val[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
val_unique

Размер валидационных данных: (10570, 5)
Количество уникальных контекстов в наборе валидационных данных: 2067


,context,context_id
0,Super Bowl 50 was an American football game to...,0
1,The Panthers finished the regular season with ...,1
2,The Broncos took an early lead in Super Bowl 5...,2
3,"CBS broadcast Super Bowl 50 in the U.S., and c...",3
4,"In early 2012, NFL Commissioner Roger Goodell ...",4
...,...,...
2062,"where is the mass of the object, is the velo...",2062
2063,A conservative force that acts on a closed sys...,2063
2064,"For certain physical scenarios, it is impossib...",2064
2065,The connection between macroscopic nonconserva...,2065
